<a href="https://colab.research.google.com/github/damilojohn/Playlist_Generator/blob/main/gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio -qU
!pip install sentence-transformers -qU

In [3]:
import transformers 
import gradio as gr 
from sentence_transformers import SentenceTransformer,util
import pickle 
import pandas as pd 
from huggingface_hub import hf_hub_download
import os 

In [4]:
pd.options.mode.chained_assignment = None  # Turn off SettingWithCopyWarning

In [5]:
from huggingface_hub import login
login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [24]:
songs_df = pd.read_csv(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename = 'spotify_transformed.csv'))
mappings = pd.read_csv(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename ='song_mappings.csv'))
verses_df = pd.read_csv(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename = 'verses.csv'))
song_embeddings = pickle.load(open(hf_hub_download('damilojohn/Personal_Playlist_Generator',repo_type='dataset',filename ='embeddings.pkl'),'rb'))

In [15]:
songs_df.head(5)

,Unnamed: 0,id,album,artist,song_id,song_name,image,url,for_lyrics
0,0,3tVQdUvClmAT7URs9V3rsp,"More Love, Less Ego",Wizkid,1DA2ADZs6O28y2rmdmpekw,2 Sugar (feat. Ayra Starr),https://i.scdn.co/image/ab67616d0000b273e944c5...,https://open.spotify.com/track/1DA2ADZs6O28y2r...,[Verse 1: Ayra Starr]\nGot a bag full of feeli...
1,1,3tVQdUvClmAT7URs9V3rsp,Made In Lagos,Wizkid,5FG7Tl93LdH117jEKYl3Cm,Essence (feat. Tems),https://i.scdn.co/image/ab67616d00001e02e944c5...,https://open.spotify.com/track/5FG7Tl93LdH117j...,"[Intro: Wizkid]\nOh-na-na, omo je ka shi ge ge..."
2,4,3tVQdUvClmAT7URs9V3rsp,Made In Lagos: Deluxe Edition,Wizkid,6jdTkoEaer7XNGSblczoSu,Mood (feat. BNXN),https://i.scdn.co/image/ab67616d00001e0290e89e...,https://open.spotify.com/track/6jdTkoEaer7XNGS...,[Intro: Buju]\nMmm-mmm\nWey come cooli my body...
3,5,3tVQdUvClmAT7URs9V3rsp,Made In Lagos,Wizkid,51mLQ3w7yR7vjdSTFLWaY5,Ginger (feat. Burna Boy),https://i.scdn.co/image/ab67616d0000485190e89e...,https://open.spotify.com/track/51mLQ3w7yR7vjdS...,[Chorus: Burna Boy & Wizkid]\nIf you want make...
4,6,3tVQdUvClmAT7URs9V3rsp,"More Love, Less Ego",Wizkid,2HYcI74UpbMMxLZ1m7Xnfn,Bad To Me,https://i.scdn.co/image/ab67616d0000b273e944c5...,https://open.spotify.com/track/2HYcI74UpbMMxLZ...,"[Verse 1]\nHiye, burst it my way\nThis kind of..."


In [36]:
mappings.head(5)

,verse,song_name
0,Got a bag full of feelings,2 Sugar (feat. Ayra Starr)
1,And a house full of regrets,2 Sugar (feat. Ayra Starr)
2,"This baggages can't fit in no plane, don't mea...",2 Sugar (feat. Ayra Starr)
3,I wanna know what's the reason why you moving ...,2 Sugar (feat. Ayra Starr)
4,As you move no dey cross my lane,2 Sugar (feat. Ayra Starr)


In [35]:
mappings.rename(columns={'Unnamed: 0':'verse','0':'song_name'},inplace=True)

In [ ]:
embedder = SentenceTransformer('msmarco-distilbert-base-v4')

In [23]:
prompt = 'I need igbo and shayo'

In [38]:
verses_df.rename(columns={'0':'verse'},inplace=True)

In [ ]:
songs_df.head(5)

In [49]:
prompt_embed = embedder.encode(prompt)
hits = util.semantic_search(prompt_embed,song_embeddings,top_k=30)
hits = pd.DataFrame.from_dict(hits[0])
verses_match = verses_df.iloc[hits['corpus_id']]
songs_match = mappings[mappings['verse'].isin(verses_match['verse'].values)]
songs_match = songs_df[songs_df['song_name'].isin(songs_match['song_name'].values)]
songs_match = songs_match.sort_values('song_name')
songs_match = songs_match.drop_duplicates(subset='song_name')
songs_name = list(songs_match['song_name'])
cover_art = list(songs_match['image'])

['https://i.scdn.co/image/ab67616d0000b27379ab73f6bfebec7c0e85f922',
 'https://i.scdn.co/image/ab67616d00001e0218d0121ff57f0b9dfbe84b54',
 'https://i.scdn.co/image/ab67616d0000b2730e4f0a913e2ea614cacd8545',
 'https://i.scdn.co/image/ab67616d0000b273395d71ee367b7ff3f66b3621',
 'https://i.scdn.co/image/ab67616d0000485181c04ab39b75f83ee9849a08',
 'https://i.scdn.co/image/ab67616d00001e02bb888232ebe4732a3e541031',
 'https://i.scdn.co/image/ab67616d0000b27398eea33da038afaa9f00f471']

In [61]:
def generate_playlist(prompt):
  prompt_embed = embedder.encode(prompt)
  hits = util.semantic_search(prompt_embed,song_embeddings,top_k=30)
  hits = pd.DataFrame.from_dict(hits[0])
  verses_match = verses_df.iloc[hits['corpus_id']]
  songs_match = mappings[mappings['verse'].isin(verses_match['verse'].values)]
  songs_match = songs_df[songs_df['song_name'].isin(songs_match['song_name'].values)]
  songs_match = songs_match.sort_values('song_name')
  songs_match = songs_match.drop_duplicates(subset='song_name')
  songs_name = list(songs_match['song_name'])
  cover_art = list(songs_match['image'])
  images = [gr.Image.update(value=art,visible=True) for art in cover_art]
  return (
      gr.Radio.update(label='songs',interactive=True,choices=songs_name),
      *images
      )
                                

In [62]:
generate_playlist(prompt)

{'choices': ['Culture',
  'Door',
  'Free Mind',
  'Last Last',
  'Logba Logba',
  'Time N Affection',
  'attention'],
 'label': 'songs',
 'show_label': None,
 'interactive': True,
 'visible': None,
 'value': <_Keywords.NO_VALUE: 'NO_VALUE'>,
 '__type__': 'update',
 'mode': 'dynamic'}

In [52]:
def set_example_prompt(examples):
  return gr.TextArea.update(value=examples[0])

In [64]:
demo = gr.Blocks()
with demo:
  gr.Markdown(
      ''' 
      # A playlist Generator for Afrobeats
      This was built using Sentence Transformers and gradio

      '''
  )
  with gr.Row():
    with gr.Column():
      gr.Markdown(
          '''
          Enter a Prompt to generate a playlist based on that prompt 
          '''
      )
      song_prompt = gr.TextArea(
          value='I need igbo and shayo',
          placeholder = "Enter a sentence that describes how you're feeling or what you want your playlist to be about"
      )
      example_prompts = gr.Dataset(
          components=[song_prompt],
          samples= [
              ['Lazy day'],
              ['I need money'],
              ['nobody gets me ']
          ]
      )
    with gr.Column():
      fetch_songs = gr.Button(value='generate your playlist',).style(full_width=True)
      with gr.Row():
                tile1 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
                tile2 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
                tile3 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
      with gr.Row():
              tile4 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
              tile5 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
              tile6 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
      with gr.Row():
              tile7 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
              tile8 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)
              tile9 = gr.Image(value="https://i.imgur.com/bgCDfT1.jpg", show_label=False, visible=True)

      # Workaround because of the Gallery issues
      tiles = [tile1, tile2, tile3, tile4, tile5, tile6, tile7, tile8, tile9]

      with gr.Row():
        song_options = gr.Radio(label='songs',interactive=True,choices=None,type='value')
    with gr.Column():
      lyrics = gr.Textbox(label='lyrics',placeholder="select a song to see it's lyrics")

  
    fetch_songs.click(
        fn=generate_playlist,
        inputs=[song_prompt],
        outputs=[song_options,tiles]
    )
    example_prompts.click(
        fn=set_example_prompt,
        inputs = [example_prompts],
        outputs = example_prompts.components
    )
  

  demo.launch(debug=False,share=True)

AttributeError: ignored